# Classifying by using top words intersections

In [52]:
import pandas as pd

df = pd.concat(
    (
        pd.read_csv('../features/metadata/tf_descriptions.csv', index_col=0),
        pd.read_csv('../features/metadata/tf_keywords.csv', index_col=0),
        #pd.read_csv('../features/metadata/tf_titles.csv', index_col=0),
    ),
    axis=1
)
df = df.groupby(df.columns, axis=1).sum() # Drop duplicate columns
df['tag'] = pd.read_csv('../features/tags.csv', index_col=0)['tag_name']

Split train-test

In [92]:
import numpy as np

train_indices = np.random.rand(len(df)) < 0.8

train = df[train_indices]
test = df[~train_indices]

In [95]:
top_words_per_tag = {
    tag: set(group.drop('tag', axis=1).sum(axis=0).nlargest(35).index)
    for tag, group in train.groupby('tag') # Notice this is done with the training set
}

In [96]:
predicted = [
    sorted(
        top_words_per_tag.items(),
        key=lambda x: len(set.intersection(set(row[row > 0].index), x[1]))
    )[-1][0]
    for _, row in test.drop('tag', axis=1).iterrows()
]

precision = sum([
        prediction == truth
        for prediction, truth in zip(predicted, test['tag'].tolist())
]) / len(predicted)

print(precision)

0.711864406779661
